In [1]:
import os
import sys
sys.path.insert(0, '..')
from utils.dataset_processing import process_data
from utils.parser import tuple_to_str

In [2]:
DATA_DIR = 'resources'
# Data dir = (1) working directory, (2) move out of test, (3) move out of src, and append to resources
DATA_DIR = os.path.join(os.getcwd(), '..', 'resources')
print("Data Directory:", DATA_DIR)

TRAIN_FILE = 'factkg_train_5k_triplets.pickle'
TEST_FILE = 'factkg_test_1k_triplets.pickle'
VALID_FILE = 'factkg_dev_300_triplets.pickle'

TRAIN_FILE_PATH = os.path.join(DATA_DIR, TRAIN_FILE)
TEST_FILE_PATH = os.path.join(DATA_DIR, TEST_FILE)
VALID_FILE_PATH = os.path.join(DATA_DIR, VALID_FILE)

train_data = None
test_data = None
valid_data = None

import pickle
# Load
with open(TRAIN_FILE_PATH, 'rb') as f:
    train_data = pickle.load(f)
with open(TEST_FILE_PATH, 'rb') as f:
    test_data = pickle.load(f)
with open(VALID_FILE_PATH, 'rb') as f:
    valid_data = pickle.load(f)

Data Directory: d:\claimpkg\claimpkg-clone\src\notebooks\..\resources


In [3]:
def get_finetune_structured_data(data: dict):
    finetune_data = []
    keys = data.keys()
    for key in keys:
        item = {"input": key, "output": ""}
        print(data[key]["triplet"])
        triple_str = [tuple_to_str(triplet) for triplet in data[key]["triplet"]]
        triple_str = "\n".join(triple_str)
        item["output"] = triple_str
        finetune_data.append(item)
        break

    return finetune_data

# train = get_finetune_structured_data(train_data)
# validation = get_finetune_structured_data(valid_data)
test = get_finetune_structured_data(test_data)

[]


In [6]:
# Export these as pickle files
with open(os.path.join(DATA_DIR, 'finetune_train_data.pickle'), 'wb') as f:
    pickle.dump(train, f)
with open(os.path.join(DATA_DIR, 'finetune_validation_data.pickle'), 'wb') as f:
    pickle.dump(validation, f)
with open(os.path.join(DATA_DIR, 'finetune_test_data.pickle'), 'wb') as f:
    pickle.dump(test, f)

# Update Trie

In [ ]:
import os
import marisa_trie
import pickle

TRIE_PATH = os.path.join(DATA_DIR, 'kg_trie.marisa')
FACT_KG_TRIE_PATH = os.path.join(DATA_DIR, 'factkg_all_distinct_entities.pickle')

trie = marisa_trie.Trie().load(TRIE_PATH)

fact_kg_entities : set = set()
with open(FACT_KG_TRIE_PATH, 'rb') as f:
    fact_kg_entities = pickle.load(f)



In [12]:
# Check for number of entities in trie and fact_kg_entities
print(f"Number of entities in trie: {len(trie)}")
print(f"Number of distinct entities in fact_kg_entities: {len(fact_kg_entities)}")

Number of entities in trie: 1349058
Number of distinct entities in fact_kg_entities: 5356


In [14]:
# Get all entities in the trie
trie_entities = set(trie.keys())
print(f"Number of distinct entities in trie_entities: {len(trie_entities)}")

Number of distinct entities in trie_entities: 1349058


In [16]:
# Rebuild the trie that contains both sets of entities
combined_entities = trie_entities.union(fact_kg_entities)
new_trie = marisa_trie.Trie(combined_entities)
print(f"Number of distinct entities in combined trie: {len(new_trie)}")
print(f"Checking the total length of orignal trie and fact_kg_entities: {len(trie) + len(fact_kg_entities)}")

Number of distinct entities in combined trie: 1349241
Checking the total length of orignal trie and fact_kg_entities: 1354414


In [17]:
# Save the new trie
NEW_TRIE_PATH = os.path.join(DATA_DIR, 'finetune_trie.marisa')
new_trie.save(NEW_TRIE_PATH)